In [9]:
import numpy as np
import pandas as pd
import random, pickle, argparse, json, os
from collections import OrderedDict
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier

In [2]:
def get_ids(json_file):
    # takes a json file with list of articles meta data including a field named 'istex_id'
    # retuns numpy array with the istex_id of the articles
    ids = np.arange(len(json_file), dtype=np.object)
    for (i,s) in enumerate(json_file):
        ids[i] = s['istex_id']
    return ids

In [3]:
def read_results(filename):
    results = pd.read_csv(filename, sep=';', index_col=0)
    sampled_results = results[results['PF'] != '  ']
    sampled_results['P'] = np.array(sampled_results['PF'], dtype=np.float)
    sampled_results['R'] = np.array(sampled_results['RM'], dtype=np.float)
    sampled_results['Score'] = (sampled_results['P'] + sampled_results['R'] )/ 2.0
    return sampled_results

In [39]:
svd = pickle.load(open('../RecSys_Exp_files/182_381_vec150_results//output_svd.pickle','rb'))
print svd.shape

(4174940, 150)


In [19]:
index = json.load(open('../RecSys_Exp_files/182_381_vec150_results/output_paragraph_index.json','rb'))

print index['0']
print index['181']
print index['182']
print index['380']
print index['381']
print index['111182']
print len(index)

UCBL_ANNOTATED001MENTAL000ROTATION10100000001
UCBL_ANNOTATED001MENTAL000ROTATION10100000162
MRISTEX_FDF8A158137E00C72623548DD41D074D8AAC1725
MRISTEX_00211AA25963CDCD086A6B844C42F42645FAD50A
ISTEX_C3D2F8606E1497B25D26FE4DE92BD00608EED309
ISTEX_17E8AB6E71469134AE441F9733B45166E4062ACF
4174940


In [15]:
for (k, v) in index.items():
    index[k] = v.split('_')[1]

print index['0']
print index['181']
print index['182']
print index['380']
print index['381']
print index['111182']
print len(index)

ANNOTATED001MENTAL000ROTATION10100000001
ANNOTATED001MENTAL000ROTATION10100000162
FDF8A158137E00C72623548DD41D074D8AAC1725
00211AA25963CDCD086A6B844C42F42645FAD50A
C3D2F8606E1497B25D26FE4DE92BD00608EED309
17E8AB6E71469134AE441F9733B45166E4062ACF
4174940


In [6]:
inv_index = json.load(open('../RecSys_Exp_files/182_381_vec150_results/output_paragraph_inversed_index.json','rb'))


In [7]:
print 'original inversed_index'
print inv_index.items()[:5]
inversed_index = dict()
for (k, v) in inv_index.items():
    key = k.split('_')[1]
    inversed_index[key] = v
print 'processed inversed_index'
print inversed_index.items()[:5]

original inversed_index
[(u'ISTEX_D89FA3AC3521074D46F4245762153DF497BFFA1F', 2002320), (u'ISTEX_18EAF4D6A126B077EB38667801D1B7292F32FF49', 2483732), (u'ISTEX_5F91044435FCC4FABB9F02E31467DCFE75F4A7BE', 1429049), (u'ISTEX_EBB03C272511D346B3D68CA10EBD929CE7A31DB8', 128349), (u'ISTEX_5776E10BEF3FF14794D60A604B2CA12B1695894B', 1225023)]
processed inversed_index
[(u'FCF1393F9B8136AC08FB67E88F94F3CF62C17288', 3517138), (u'482E1102A1114327A744FD2ADB4D9F8FF7E9A70B', 751643), (u'A81022B6295AE66F68A10222C3B94A06B033C1BA', 3983232), (u'F27F00C86A2994208F0143A5CD2217AB87B21A64', 1906438), (u'939B5153F5A4B8053FFAF8E7BCE062B341DB9BEB', 3603049)]


In [8]:
#double-check index values by length
print 'id lenght must be:', len(index['182'])
print "exceptions if any:"
for (k, v) in index.items():
    if len(v) < 40:
        print (k, v)

id lenght must be: 40
exceptions if any:


In [4]:
seed = json.load(open('../RecSys_Exp_files/ISTEX_MentalRotation/sample_data/sportArticlesAsIstex_UniqID_182.json','r'))
test = json.load(open('../RecSys_Exp_files/ISTEX_MentalRotation/sample_data/MentalRotationInMetaDataIstexWithoutAnnotated.json','r'))
seed_ids = get_ids(seed)
test_ids = get_ids(test)

In [9]:
print seed_ids[0]
print test_ids[0]

ANNOTATED001MENTAL000ROTATION10100000001
FDF8A158137E00C72623548DD41D074D8AAC1725


In [11]:
results = read_results('annotations/SSbE.csv')
# considering that the good result is the one that at least one of the experts marked as relevant
#  while the other could not decide
good_results = results[results['Score'] > 0.5]["Articles"].values
# considering that the bad result is the one that at least one of the experts marked as irrelevant
bad_results = results[results['Score'] < 0.5]["Articles"].values
# Results with disagreement between the 2 experts or both cannot_decide were excluded (0.5 values)
excluded_results = results[results['Score'] == 0.5]["Articles"].values

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [10]:
def get_ids_from_link(results):
    # takes a numpy array of url_link values
    # retuns numpy array with the istex_id of the articles
    result_ids = np.arange(results.shape[0], dtype=np.object)
    for (i,r) in enumerate(results):
        splt = r.split('/')
        result_ids[i] = splt[4]
    return result_ids

In [12]:
good_ids = get_ids_from_link(good_results)
bad_ids = get_ids_from_link(bad_results)
excluded_ids = get_ids_from_link(excluded_results)
print len(good_ids)
len(excluded_ids)

21


15

In [13]:
for doc_id in good_ids:
    if len(doc_id) != 40:
        print doc_id

for doc_id in seed_ids:
    if len(doc_id) != 40:
        print doc_id
        
for doc_id in test_ids:
    if len(doc_id) != 40:
        print doc_id

In [14]:
print random.sample(range(20), 10)
indecies = np.random.choice(range(4174940),size=5)
indecies.tolist()

[9, 8, 7, 19, 5, 16, 6, 14, 0, 1]


[327183, 1684624, 3904508, 3821319, 1108301]

def balanced_activelearn_ids_dataset(pos_arr, neg_arr, others_arr, size):
    pos_size = size/2
    neg_size = (size - pos_size) - len(neg_arr) # in case of odd int value of size
    positives = sampling(pos_arr, 'positive', pos_size)
    negatives_sure = sampling(neg_arr, 'negative', len(neg_arr))
    negatives_rand = sampling(others_arr, 'negative', neg_size)
    res = np.hstack((positives, negatives_sure))
    return np.hstack((res, negatives_rand))

In [15]:
negative_ids = bad_ids
negative_ids_lst = negative_ids.tolist()
positive_ids_1= np.hstack((good_ids, seed_ids))
positive_ids = np.hstack((positive_ids_1, test_ids))
positive_ids_lst = positive_ids.tolist()
print 'size of negatives:', len(negative_ids_lst)
print 'size of positives:', len(positive_ids_lst)

size of negatives: 70
size of positives: 402


In [16]:
all_ids = np.hstack((negative_ids_lst, positive_ids_lst))
all_ids_lst = all_ids.tolist()
others_size = len(index.keys()) - len(all_ids_lst)

other_ids = np.arange(others_size-181, dtype=np.object)#This extra 181 are a bug that need debugging
i = 0
for (k, v) in index.items():
    if v in all_ids_lst:
        continue
    else: 
        other_ids[i] = v
        i += 1
print other_ids[-10:]
other_ids_lst = other_ids.tolist()

[u'B4B4F0628AFF81062EB596FFC0DD6A637F27737E'
 u'A5F0E06E96CB5ECC5E3C7A376F9A006C2B21EDCC'
 u'E1F652D41E8E824FAC586C990F03033637F5871E'
 u'89EF426699BD2C1085F4B1BFD80CCE098301EED4'
 u'078EB70E4B0CF7579CEA897DAA7B9E834A988103'
 u'2E15B0334C74F74F2F129D2A8B1264BFF1C2C658'
 u'B4BCF54DB5ECD8392CD6D07E62A60931AB27DC30'
 u'D4D87F6F661A02E2041D2414AE520547A3B23200'
 u'36F76E60FFDB3860F06EB0B379552F74BE639CF4'
 u'D25A0773984B9E38EBBEE50D006738FB61A2A857']


In [17]:
def get_svd_from_doc_id(ids_lst):
    res = np.arange(len(ids_lst), dtype=np.object)
    i = 0
    for (i, idd) in enumerate(ids_lst):
        res[i] = svd[int(inversed_index[idd])]

    return res

In [40]:
positive_docs_svd = get_svd_from_doc_id(positive_ids_lst)
negative_docs_svd = get_svd_from_doc_id(negative_ids_lst)

In [23]:
#We have an issue using this function...
#dataset_ids = balanced_activelearn_ids_dataset(positive_ids, negative_ids, others, 750)
#dataset_ids[-306:]
#len(dataset_ids)

In [56]:
#generate some randome negative examples to have a balanced dataset for training
size_of_positives = len(positive_docs_svd)
print 'size_of_positives', size_of_positives
size_of_negatives = len(negative_docs_svd)
print 'size_of_negatives', size_of_negatives
neg_random_size = size_of_positives - size_of_negatives
print 'needed size of negative random sampled documents:', neg_random_size
indicies = random.sample(range(len(index)), neg_random_size)
neg_random = other_ids[indicies]
neg_random_lst = neg_random.tolist()
negative_rand_docs_svd = get_svd_from_doc_id(neg_random_lst)
print 'size of generated negative_rand_docs_svd:', len(negative_rand_docs_svd)

size_of_positives 402
size_of_negatives 70
needed size of negative random sampled documents: 332
size of generated negative_rand_docs_svd: 332


TODO: Now we have 384 positive examples and 70+314 negative examples, use these for training the classifier 
and finally use that classifier to get topn results

In [57]:
len(positive_docs_svd[0])

150

In [18]:
def generate_dataset(positive_docs_svd, negative_docs_svd, negative_rand_docs_svd):

    #Takes 3 different size arrays of type numpy_array
    #First array represent the positive examples while the others are negative examples
    #Returns a dataset (X,y) where X is a combination of the 3 arrays and y is the lables
    vector_size = len(positive_docs_svd[0])
    ones_lables = np.ones(len(positive_docs_svd), dtype=np.int)
    size_negative = len(negative_docs_svd) + len(negative_rand_docs_svd)
    negative_examples = np.hstack((negative_docs_svd, negative_rand_docs_svd))
    zeros_lables = np.zeros(len(negative_examples), dtype=np.int)
    y = np.hstack((ones_lables, zeros_lables))
    X_in = np.hstack((positive_docs_svd, negative_examples))
    
    X = np.zeros((len(X_in),vector_size))
    for (i, x) in enumerate(X_in):
        for (j, f) in enumerate(x):
            X[i,j] = f

    return X, y

In [68]:
#Process needs about 20 minutes on 32GB-RAM didicated machine with swapping memory of 90GB
#We recommend a virtual machine with ~90GB-RAM instead or at least 64GB with swapping enabled
classifier = RandomForestClassifier(n_estimators=500)
X, y = generate_dataset(positive_docs_svd, negative_docs_svd, negative_rand_docs_svd)


print X.shape
print y.shape
classifier.fit(X, y)
sim_results = classifier.predict_proba(svd)

pickle.dump(sim_results, open('results/activelearn_sim_results.pickle', 'wb'))
pickle.dump(classifier, open('activelearn_clf.pickle', 'wb'))

(804, 150)
(804,)


In [20]:
sim_results = pickle.load(open('results/activelearn_sim_results.pickle', 'rb'))
print sim_results.shape
print 'random example', sim_results[3405154]
print 'positive example', sim_results[int(inversed_index[positive_ids[200]])]
print 'negative example', sim_results[int(inversed_index[negative_ids[50]])]

(4174940, 2)
random example [ 0.882  0.118]
positive example [ 0.038  0.962]
negative example [ 0.812  0.188]


In [21]:
sim_results[:,1][int(inversed_index[positive_ids[200]])]

0.96199999999999997

In [22]:
index['181']

u'ANNOTATED001MENTAL000ROTATION10100000162'

In [3]:
sim_results = pickle.load(open('results/activelearn_sim_results.pickle', 'rb'))

In [4]:
def sorted_results(sim_results, doc_id_dict, count_positive):
    results = dict()
    for i, score in enumerate(sim_results[:,1][count_positive:]):
        doc_id = doc_id_dict[str(count_positive + i)]
        results[doc_id] = score
    sorted_results = OrderedDict(sorted(results.items(), key=lambda k: k[1], reverse=True))
    return sorted_results

In [20]:
results = sorted_results(sim_results, index, 182)
print len(results)
len(sim_results) - len(results)

4174758


182

In [10]:
top10k = pd.DataFrame(results.items()[:10000])
top10k.to_csv('top10k_activelearn.csv')

In [11]:
results_excluding_MRISTEX = sorted_results(sim_results, index, 381)
print len(results_excluding_MRISTEX)
len(sim_results) - len(results_excluding_MRISTEX)

4174559


381

In [12]:
len(results) - len(results_excluding_MRISTEX)

199

In [40]:
len(sim_results)

4174940

In [9]:
len(sim_results) - len(results)

182

In [49]:
len(sim_results) - len(results_excluding_MRISTEX)

381

In [13]:
top10k_results_excluding_MRISTEX = pd.DataFrame(results_excluding_MRISTEX.items()[:10000])
top10k_results_excluding_MRISTEX.to_csv('top10k_activelearn_excluding_MRISTEX.csv')

In [14]:
top10k_results_excluding_MRISTEX[:10]

,0,1
0,ISTEX_7E34BEBFB13F51AEC21E8DBA16B56F69801FB6CD,0.998
1,ISTEX_0552C8637274B37867899F4D2783BCC00E438A6C,0.988
2,ISTEX_4CC2253534C9F032393A3971E6CC5E4AAD845A05,0.988
3,ISTEX_B18087285607420B2686431EDD704EAFD76D5F5B,0.986
4,ISTEX_94B114260B0B11B0D8BBDDFBE4E186D0A14282DD,0.986
5,ISTEX_1A76D96B351A7BC76575D8B96C1A8A67655409BC,0.984
6,ISTEX_BFD567F907BD7F30E5FBE3886455092F3AD6AC73,0.984
7,ISTEX_D1FF7B2B1B79430FCD64F71DA33C0B275523E886,0.980
8,ISTEX_218785326AB46691F45DA7216E0E1C97890FE019,0.980
9,ISTEX_D964FD657961F0B54BDF142FB8D34C1C2E6C54C8,0.980


In [48]:
for res_doc in results_excluding_MRISTEX.items()[:10]:
    print res_doc


(u'65F3E7E331C9C071195EE30ACEFFACA01F27D8CD', 0.96399999999999997)
(u'02DFD687BB59004E10F87F8A1B8B29D8D1643DD9', 0.96399999999999997)
(u'F25927D9CCB2A1DDC5045FA4388C33D090940C5D', 0.96399999999999997)
(u'B9DDBDA6C785B111C56033CA730F956EE527CA02', 0.96199999999999997)
(u'4A8A47F73791EA4109EBEC2CDDB8B2B8E6A44D38', 0.96199999999999997)
(u'9F3A198CFC85E54C3644098D97BE85CB64A4F86E', 0.96199999999999997)
(u'7116DEF8FC765D329BAA1C826D32803360D1217F', 0.96199999999999997)
(u'401A0E69996285F9ACE0CD1A5AB73D25960F5959', 0.95999999999999996)
(u'7ED5E3762DE21EEB9E9CC231447D6F32BBF9BBB3', 0.95999999999999996)
(u'FCCE2ABACF816D7071126DF3D3500AB0F6BF156E', 0.95799999999999996)


In [33]:
for (i, res_doc) in enumerate(results.items()):
    print res_doc
    if i > 10:
        break

(u'7E34BEBFB13F51AEC21E8DBA16B56F69801FB6CD', 0.998)
(u'F4CDE81D8D8F23F6507262F6574A6F6E85F2C82D', 0.99199999999999999)
(u'0552C8637274B37867899F4D2783BCC00E438A6C', 0.98799999999999999)
(u'4CC2253534C9F032393A3971E6CC5E4AAD845A05', 0.98799999999999999)
(u'94B114260B0B11B0D8BBDDFBE4E186D0A14282DD', 0.98599999999999999)
(u'B18087285607420B2686431EDD704EAFD76D5F5B', 0.98599999999999999)
(u'1A76D96B351A7BC76575D8B96C1A8A67655409BC', 0.98399999999999999)
(u'BFD567F907BD7F30E5FBE3886455092F3AD6AC73', 0.98399999999999999)
(u'218785326AB46691F45DA7216E0E1C97890FE019', 0.97999999999999998)
(u'D1FF7B2B1B79430FCD64F71DA33C0B275523E886', 0.97999999999999998)
(u'D964FD657961F0B54BDF142FB8D34C1C2E6C54C8', 0.97999999999999998)
(u'622C8F010109933F69BC38E537A1FFFFF307E9FA', 0.97799999999999998)


In [2]:
pickle.dump(results, open('results/activelearn_results.pickle', 'wb'))

NameError: name 'results' is not defined

In [35]:
pickle.dump(results.items()[:10000], open('results/activelearn_top10k_results.pickle', 'wb'))

In [2]:
df = pd.DataFrame()
top10k = df.from_csv('top10k_activelearn.csv')
top_ids = [article.split('_')[1] for article in top10k['0']]
mr_articles = json.load(open('sample_data/MentalRotationInMetaDataIstexWithoutAnnotated.json','r'))
mr_ids = [doc['istex_id'] for doc in mr_articles]

In [3]:
top_wo_mr = []
for top_id in top_ids:
    if  top_id not in mr_ids:
        top_wo_mr.append(top_id)
print 'number of results without "mental rotation" in meta data out of the top 10000 = ',len(top_wo_mr)

number of results without "mental rotation" in meta data out of the top 10000 =  9620


In [12]:
top959 = pd.DataFrame(top_wo_mr[:959])
top959.to_csv('top959_activelearn.csv')

In [14]:
top10kwo = pd.DataFrame(top_wo_mr[:10000])
top10kwo.to_csv('top10kwo_activelearn.csv')